# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
print(city_data_df.columns)

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')


In [11]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City", 
)


# Display the map
hvplot.save(humidity_map, "output_data/humidity_map.html")
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
cities_narrow_lats_df = city_data_df[(city_data_df['Lat'] >= -10) & (city_data_df['Lat'] <= 50)]

# Drop any rows with null values
ideal_weather_city_df = cities_narrow_lats_df.dropna()
# Display sample data
ideal_weather_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,-251.09,35,21,2.60,SA,1666108228
3,3,jamestown,42.0970,-79.2353,-267.38,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,-258.62,48,59,1.20,CN,1666108230
6,6,albany,42.6001,-73.9662,-262.79,74,29,2.10,US,1666108231
12,12,trairi,-3.2778,-39.2689,-242.08,51,34,8.09,BR,1666108038
...,...,...,...,...,...,...,...,...,...,...
565,565,wyndham,37.6985,-77.6125,-260.34,52,3,1.79,US,1666108480
567,567,sangamner,19.5667,74.2167,-249.33,85,100,2.28,IN,1666108559
568,568,road town,18.4167,-64.6167,-242.13,70,40,4.12,VG,1666108560
569,569,ranong,9.9658,98.6348,-248.81,93,100,1.44,TH,1666108330


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city_df[['City', 'Country','Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''


# Display sample data
hotel_df.head()

C:\Users\Rich\AppData\Local\Temp\ipykernel_23508\3280282049.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,
6,albany,US,42.6001,-73.9662,74,
12,trairi,BR,-3.2778,-39.2689,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
    
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key  
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
lanzhou - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
trairi - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
nuevo laredo - nearest hotel: No hotel found
senneterre - nearest hotel: No hotel found
zangakatun - nearest hotel: No hotel found
saint-louis - nearest hotel: No hotel found
midwest city - nearest hotel: No hotel found
naberera - nearest hotel: No hotel found
drochia - nearest hotel: No hotel found
labuhan - nearest hotel: No hotel found
tomatlan - nearest hotel: No hotel found
collierville - nearest hotel: No hotel found
kapaa - nearest hotel: No hotel found
mogadishu - nearest hotel: No hotel found
tsalka - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
businga - nearest hotel: No hotel found
lasa - nearest hotel: No hotel found
cayenne - nearest hotel: No hotel found
airai - nearest hotel: No hotel foun

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
3,jamestown,US,42.0970,-79.2353,77,No hotel found
4,lanzhou,CN,36.0564,103.7922,48,No hotel found
6,albany,US,42.6001,-73.9662,74,No hotel found
12,trairi,BR,-3.2778,-39.2689,51,No hotel found
...,...,...,...,...,...,...
565,wyndham,US,37.6985,-77.6125,52,No hotel found
567,sangamner,IN,19.5667,74.2167,85,No hotel found
568,road town,VG,18.4167,-64.6167,70,No hotel found
569,ranong,TH,9.9658,98.6348,93,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Lat",
    scale = 1,
    color = "City", 
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
hvplot.save(hotel_map, "output_data/hotel_map.html")
hotel_map 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)